In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,513 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,188 kB]
Get:13 http://archive.ubuntu.c

In [3]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

In [4]:
# Gán quyền cho thư mục "bin" để hệ thống được quyền sử dụng các tập tin trong đó
!chmod +x /content/gdrive/MyDrive/Colab\ Notebooks/Spark/spark-3.5.1-bin-hadoop3/bin/*

In [5]:
# Tạo biến môi trường
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Khai báo đường dẫn đến thư mục đã giải nén
os.environ["SPARK_HOME"] = "/content/gdrive/MyDrive/Colab Notebooks/Spark/spark-3.5.1-bin-hadoop3"

In [6]:
!pip install -q findspark

In [7]:
import findspark
findspark.init()

In [8]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf

In [9]:
SparkContext.setSystemProperty('spark.executor.memory', '12g')
sc = SparkContext(master='local', appName='Collaborative_filtering')

In [10]:
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession(sc)

In [12]:
file_path = "/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/data/Danh_gia_new.csv"

In [13]:
data = spark.read.csv(file_path, inferSchema=True, header=True)

In [14]:
data.show(100,truncate=True)

+---+-------------+--------------------+--------------+-------------------+------+-----------+
| id|ma_khach_hang|  noi_dung_binh_luan|ngay_binh_luan|      gio_binh_luan|so_sao|ma_san_pham|
+---+-------------+--------------------+--------------+-------------------+------+-----------+
|  1|          443|sử_dụng_dàng thoả...|    29/04/2023|2024-12-04 17:06:00|     5|  308500015|
|  2|         1030|sử_dụng dãng thoả...|    30/04/2023|2024-12-04 15:04:00|     5|  308500015|
|  3|          689|thích hasaki sản_...|    30/04/2023|2024-12-04 18:34:00|     5|  422216594|
|  4|         2519|sản_phẩm khả_năng...|    17/07/2022|2024-12-04 13:48:00|     5|  204100075|
|  5|          402|sữa rửa mặt tốt s...|    15/04/2023|2024-12-04 23:04:00|     5|  422208977|
|  7|         2517|dòng srm chân tuy...|    25/12/2022|2024-12-04 17:11:00|     5|  204100019|
|  8|         1750|sài bha obagi đổi...|      9/7/2022|2024-12-04 04:34:00|     5|  100220035|
|  9|         3394|                 xài|      7/7/

In [15]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- ma_khach_hang: integer (nullable = true)
 |-- noi_dung_binh_luan: string (nullable = true)
 |-- ngay_binh_luan: string (nullable = true)
 |-- gio_binh_luan: timestamp (nullable = true)
 |-- so_sao: integer (nullable = true)
 |-- ma_san_pham: integer (nullable = true)



In [16]:
data_sub = data.select(['ma_san_pham', 'so_sao', 'ma_khach_hang'])

In [17]:
data_sub.count()

19629

In [18]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import isnan, when, count, col, udf

In [19]:
data_sub = data_sub.withColumn("so_sao", data_sub["so_sao"].cast(DoubleType()))

In [20]:
data_sub.printSchema()

root
 |-- ma_san_pham: integer (nullable = true)
 |-- so_sao: double (nullable = true)
 |-- ma_khach_hang: integer (nullable = true)



In [21]:
data_sub.show(100, truncate=True)

+-----------+------+-------------+
|ma_san_pham|so_sao|ma_khach_hang|
+-----------+------+-------------+
|  308500015|   5.0|          443|
|  308500015|   5.0|         1030|
|  422216594|   5.0|          689|
|  204100075|   5.0|         2519|
|  422208977|   5.0|          402|
|  204100019|   5.0|         2517|
|  100220035|   5.0|         1750|
|  204100024|   5.0|         3394|
|  422216354|   4.0|         1817|
|  100250032|   5.0|         3556|
|  422205163|   5.0|         2375|
|  249500019|   5.0|         2027|
|  422209718|   5.0|         2928|
|  413400001|   2.0|         3059|
|  422204884|   5.0|         1118|
|  422208183|   4.0|         2273|
|  100230056|   5.0|          736|
|  100230056|   5.0|          731|
|  422205637|   4.0|         4234|
|  422203776|   5.0|         2696|
|  204900005|   5.0|         1281|
|  351700026|   5.0|         3417|
|  204900056|   5.0|         1268|
|  422208328|   5.0|         1360|
|  357100011|   5.0|         2255|
|  422203300|   2.0|

In [22]:
data_sub.select([count(when(col(c).isNull(), c)).alias(c) for c in
           data_sub.columns]).toPandas().T

,0
ma_san_pham,0
so_sao,0
ma_khach_hang,0


In [23]:
# Phân biệt mã người dùng và mã sản phẩm
users = data_sub.select("ma_khach_hang").distinct().count()
products = data_sub.select("ma_san_pham").distinct().count()
numerator = data_sub.count()

In [24]:
display(numerator, users, products)

19629

4208

682

In [25]:
# Số lượng ma trận xếp hạng có thể chứa nếu không có ô trống
denominator = users * products
denominator

2869856

In [26]:
#Tính độ thưa
sparsity = 1 - (numerator*1.0 / denominator)
print ("Sparsity: "), sparsity

Sparsity: 


(None, 0.9931602840003122)

## Thuật toán Đề xuất ALS

In [27]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [28]:
# Chuyển đổi chuỗi thành chỉ số
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

In [29]:
# Tạo chỉ số
indexer = StringIndexer(inputCol='ma_san_pham',
                        outputCol='ma_san_pham_idx')

# Tạo định danh các hạng mục bằng chỉ số
indexer_model = indexer.fit(data_sub)

# Tạo thêm cột chỉ số
data_indexed = indexer_model.transform(data_sub)

# Lặp lại quá trình trên cho thuộc tính phân loại khác
indexer1 = StringIndexer(inputCol='ma_khach_hang',
                         outputCol='ma_khach_hang_idx')
indexer1_model = indexer1.fit(data_indexed)
data_indexed = indexer1_model.transform(data_indexed)

In [30]:
data_indexed.show(100, truncate=True)

+-----------+------+-------------+---------------+-----------------+
|ma_san_pham|so_sao|ma_khach_hang|ma_san_pham_idx|ma_khach_hang_idx|
+-----------+------+-------------+---------------+-----------------+
|  308500015|   5.0|          443|           10.0|            191.0|
|  308500015|   5.0|         1030|           10.0|            164.0|
|  422216594|   5.0|          689|           23.0|             39.0|
|  204100075|   5.0|         2519|          581.0|            415.0|
|  422208977|   5.0|          402|           88.0|            190.0|
|  204100019|   5.0|         2517|           20.0|           1984.0|
|  100220035|   5.0|         1750|           36.0|           1474.0|
|  204100024|   5.0|         3394|          357.0|           2711.0|
|  422216354|   4.0|         1817|          454.0|           2437.0|
|  100250032|   5.0|         3556|          280.0|           2136.0|
|  422205163|   5.0|         2375|           12.0|            324.0|
|  249500019|   5.0|         2027|

In [31]:
data_indexed.select([count(when(col(c).isNull(), c)).alias(c) for c in
           data_indexed.columns]).toPandas().T

,0
ma_san_pham,0
so_sao,0
ma_khach_hang,0
ma_san_pham_idx,0
ma_khach_hang_idx,0


In [32]:
# Tách dataset theo tỷ lệ 80/20
(training, test) = data_indexed.randomSplit([0.8, 0.2])

In [33]:
# Tạo mô hình ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [34]:
als = ALS(maxIter=10,
          regParam=0.09,
          rank = 25,
          userCol="ma_khach_hang_idx",
          itemCol="ma_san_pham_idx",
          ratingCol="so_sao",
          coldStartStrategy="drop",
          nonnegative=True)
model = als.fit(training)

In [35]:
# Đánh giá mô hình bằng RMSE trên tập dữ liệu đánh giá.
predictions = model.transform(test)

In [36]:
predictions.show(100)

+-----------+------+-------------+---------------+-----------------+----------+
|ma_san_pham|so_sao|ma_khach_hang|ma_san_pham_idx|ma_khach_hang_idx|prediction|
+-----------+------+-------------+---------------+-----------------+----------+
|  100220037|   4.0|         3672|           77.0|           1088.0| 3.9382367|
|  204900128|   5.0|         1433|           46.0|            833.0|  4.931531|
|  223600039|   1.0|         4072|          315.0|           2866.0| 0.9894612|
|  223600048|   5.0|         3590|          296.0|           2142.0| 3.8042116|
|  225200013|   5.0|         2169|          161.0|            148.0| 4.8120747|
|  225200015|   5.0|         2169|          174.0|            148.0| 4.8598924|
|  248100018|   5.0|         2240|          376.0|           1591.0| 2.9214163|
|  308500009|   5.0|         2449|           13.0|            463.0| 4.9101453|
|  308500037|   5.0|         2449|           15.0|            463.0|  4.920524|
|  318900012|   5.0|          100|      

In [37]:
evaluator = RegressionEvaluator(metricName="rmse",
                                labelCol="so_sao",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

In [38]:
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.6761996036236866


In [40]:
# save model to disk
model.save("/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/Collaborative_ALS_model_v1")

## Tạo đề xuất sản phẩm cho tất cả khách hàng

In [41]:
# Lấy 5 đề xuất sản phẩm có chỉ số đánh giá cao nhất.
user_recs = model.recommendForAllUsers(5)

In [42]:
user_recs.show(100, truncate=False)

+-----------------+------------------------------------------------------------------------------------------+
|ma_khach_hang_idx|recommendations                                                                           |
+-----------------+------------------------------------------------------------------------------------------+
|0                |[{513, 5.442994}, {515, 5.3534055}, {514, 5.3276534}, {139, 5.2125106}, {140, 5.1619134}] |
|1                |[{270, 5.421175}, {271, 5.390791}, {365, 5.348533}, {493, 5.3404775}, {406, 5.2832675}]   |
|2                |[{393, 6.349401}, {271, 6.030382}, {270, 6.0150657}, {406, 5.988628}, {389, 5.935176}]    |
|3                |[{588, 6.300522}, {406, 6.2733765}, {218, 6.130953}, {257, 5.976953}, {242, 5.9682093}]   |
|4                |[{588, 5.778089}, {515, 5.506387}, {270, 5.5026827}, {514, 5.500013}, {271, 5.4933844}]   |
|5                |[{588, 6.431342}, {270, 6.312698}, {271, 6.287999}, {513, 6.205111}, {406, 6.190081}]     |
|

In [43]:
user_recs.count()

3940

In [44]:
for user in user_recs.head(5):
    print(user)
    print("\n")

Row(ma_khach_hang_idx=0, recommendations=[Row(ma_san_pham_idx=513, rating=5.442994117736816), Row(ma_san_pham_idx=515, rating=5.353405475616455), Row(ma_san_pham_idx=514, rating=5.327653408050537), Row(ma_san_pham_idx=139, rating=5.212510585784912), Row(ma_san_pham_idx=140, rating=5.1619133949279785)])


Row(ma_khach_hang_idx=1, recommendations=[Row(ma_san_pham_idx=270, rating=5.421175003051758), Row(ma_san_pham_idx=271, rating=5.390790939331055), Row(ma_san_pham_idx=365, rating=5.3485331535339355), Row(ma_san_pham_idx=493, rating=5.340477466583252), Row(ma_san_pham_idx=406, rating=5.283267498016357)])


Row(ma_khach_hang_idx=2, recommendations=[Row(ma_san_pham_idx=393, rating=6.349400997161865), Row(ma_san_pham_idx=271, rating=6.03038215637207), Row(ma_san_pham_idx=270, rating=6.015065670013428), Row(ma_san_pham_idx=406, rating=5.988627910614014), Row(ma_san_pham_idx=389, rating=5.935175895690918)])


Row(ma_khach_hang_idx=3, recommendations=[Row(ma_san_pham_idx=588, rating=6.30052185

## Lưu thành file

In [45]:
from time import time

In [46]:
t0 = time()

In [47]:
df_khach_hang_khach_hang_id = data_indexed.select('ma_khach_hang_idx', 'ma_khach_hang').distinct()

In [48]:
df_khach_hang_khach_hang_id.count()

4208

In [49]:
df_khach_hang_khach_hang_id.show(100)

+-----------------+-------------+
|ma_khach_hang_idx|ma_khach_hang|
+-----------------+-------------+
|            313.0|         1268|
|              6.0|          182|
|              0.0|          792|
|           1256.0|         2107|
|           2061.0|          300|
|            232.0|         2368|
|           1424.0|         1270|
|            210.0|         2329|
|           1941.0|         2133|
|           2722.0|         3448|
|            865.0|         1485|
|            917.0|          379|
|            683.0|          197|
|           2909.0|         4225|
|           1826.0|         1302|
|           2953.0|         4361|
|           1792.0|         4239|
|           2776.0|         3680|
|            602.0|          786|
|           1511.0|         2003|
|           1833.0|         1318|
|            872.0|         1498|
|           3775.0|         3903|
|           1908.0|         1830|
|           2772.0|         3635|
|           4048.0|         4463|
|           24

In [50]:
df_san_pham_san_pham_idx = data_indexed.select('ma_san_pham_idx', 'ma_san_pham').distinct()

In [51]:
df_san_pham_san_pham_idx.count()

682

In [52]:
df_san_pham_san_pham_idx.show(100)

+---------------+-----------+
|ma_san_pham_idx|ma_san_pham|
+---------------+-----------+
|          186.0|  365000019|
|          614.0|  422216605|
|          242.0|  422204775|
|           77.0|  100220037|
|          451.0|  422214134|
|          567.0|  422202421|
|          631.0|  318900010|
|           11.0|  422202253|
|          472.0|  100230026|
|          143.0|  422204657|
|          233.0|  418800004|
|          296.0|  223600048|
|          618.0|  100230076|
|          170.0|  422216098|
|          411.0|  422205806|
|          196.0|  100180113|
|          272.0|  248700075|
|          448.0|  422212892|
|          228.0|  200500293|
|          459.0|  422218099|
|          210.0|  422208910|
|          244.0|  226200018|
|           19.0|  422206537|
|          488.0|  422202216|
|          217.0|  214000001|
|          288.0|  422209047|
|          650.0|  422209198|
|            0.0|  422204259|
|          108.0|  223600009|
|           49.0|  422215804|
|         

In [53]:
new_user_recs = user_recs.join(df_khach_hang_khach_hang_id, on=['ma_khach_hang_idx'], how='left')

In [54]:
new_user_recs.show(100, truncate=False)

+-----------------+------------------------------------------------------------------------------------------+-------------+
|ma_khach_hang_idx|recommendations                                                                           |ma_khach_hang|
+-----------------+------------------------------------------------------------------------------------------+-------------+
|0                |[{513, 5.442994}, {515, 5.3534055}, {514, 5.3276534}, {139, 5.2125106}, {140, 5.1619134}] |792          |
|1                |[{270, 5.421175}, {271, 5.390791}, {365, 5.348533}, {493, 5.3404775}, {406, 5.2832675}]   |1157         |
|2                |[{393, 6.349401}, {271, 6.030382}, {270, 6.0150657}, {406, 5.988628}, {389, 5.935176}]    |220          |
|3                |[{588, 6.300522}, {406, 6.2733765}, {218, 6.130953}, {257, 5.976953}, {242, 5.9682093}]   |121          |
|4                |[{588, 5.778089}, {515, 5.506387}, {270, 5.5026827}, {514, 5.500013}, {271, 5.4933844}]   |1109         |


In [55]:
new_user_recs.count()

3940

In [56]:
# Lưu kết quả huấn luyện thành Collaborative_ALS_U.parquet và Collaborative_ALS_P.parquet, một cái cho khách hàng, một cái cho sản phảm.
new_user_recs.write.parquet('/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/Collaborative_ALS_U_v1.parquet', mode='overwrite')
df_san_pham_san_pham_idx.write.parquet('/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/Collaborative_ALS_P_v1.parquet', mode='overwrite')

In [57]:
time_duration = time() - t0
print(time_duration)

31.954134225845337


## Đọc file San_pham.csv

In [58]:
path = "/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/data/San_pham.csv"

In [59]:
products = spark.read.csv(
    path,
    inferSchema=True,  # Tự động suy đoán kiểu dữ liệu
    header=True,       # Đọc dòng đầu tiên làm tiêu đề cột
    sep=',',           # Định nghĩa dấu phân cách, thay đổi nếu cần (vd: ';')
    quote='"',         # Định nghĩa ký tự bao chuỗi (nếu dữ liệu chứa dấu phân cách bên trong chuỗi)
    escape='\\',       # Ký tự để thoát ký tự đặc biệt
    multiLine=True,    # Đọc dữ liệu nhiều dòng (nếu chuỗi bị xuống dòng)
    ignoreTrailingWhiteSpace=True,  # Bỏ khoảng trắng ở cuối
    ignoreLeadingWhiteSpace=True    # Bỏ khoảng trắng ở đầu
)

In [60]:
products.show(100,truncate=True)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|         ma_san_pham|        ten_san_pham|             gia_ban|             gia_goc|           phan_loai|               mo_ta|diem_trung_binh|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+
|           318900012|Nước Hoa Hồng Kla...|              209000|              435000|2x180ml\n180ml\nK...|Nước Hoa Hồng Kla...|            4.8|
|           205100137|Nước Tẩy Trang L'...|              147000|              229000|2x400ml\n95ml\n40...|Nước Tẩy Trang L'...|            4.7|
|           422208973|Sữa Rửa Mặt CeraV...|              343000|              455000|88ml\n236ml\n473m...|Sữa Rửa Mặt Cerav...|            4.9|
|           204900013|Kem Chống Nắng La...|              377000|              560000|        2x50ml\n50ml|Kem chống nắng gi...|         

In [61]:
products_sub = products.select(['ma_san_pham', 'ten_san_pham', 'mo_ta'])

In [62]:
products_sub.show(100,truncate=True)

+--------------------+--------------------+--------------------+
|         ma_san_pham|        ten_san_pham|               mo_ta|
+--------------------+--------------------+--------------------+
|           318900012|Nước Hoa Hồng Kla...|Nước Hoa Hồng Kla...|
|           205100137|Nước Tẩy Trang L'...|Nước Tẩy Trang L'...|
|           422208973|Sữa Rửa Mặt CeraV...|Sữa Rửa Mặt Cerav...|
|           204900013|Kem Chống Nắng La...|Kem chống nắng gi...|
|           253900006|Kem Chống Nắng Sk...|Kem Chống Nắng Sk...|
|           200400003|Nước Tẩy Trang Bi...|Nước Tẩy Trang Bi...|
|           422218358|Sữa Chống Nắng An...|Sữa Chống Nắng An...|
|           200400004|Nước Tẩy Trang Bi...|Nước Tẩy Trang Bi...|
|           422210400|Kem Chống Nắng L'...|UV Defender SPF 5...|
|           205100146|Nước Tẩy Trang L'...|Nước Tẩy Trang L'...|
|           422218359|Sữa Chống Nắng An...|Sữa Chống Nắng An...|
|           422204795|Gel Rửa Mặt Cosrx...|Gel Rửa Mặt Cosrx...|
|           422200236|Kem

## Nhập ma_khach_hang để tìm gợi ý sản phẩm từ những khách hàng khác

In [63]:
def get_recommendations_with_details():
    while True:
        try:
            # Nhập mã khách hàng
            ma_khach_hang = input("Nhập mã khách hàng (hoặc nhập 0 để thoát): ")
            ma_khach_hang = int(ma_khach_hang)  # Kiểm tra số nguyên

            if ma_khach_hang == 0:
                print("Thoát chương trình.")
                break  # Thoát khỏi vòng lặp

            # Tìm kiếm gợi ý sản phẩm
            find_user_rec = new_user_recs.filter(new_user_recs['ma_khach_hang'] == str(ma_khach_hang))
            user = find_user_rec.first()

            if user is None:
                print(f"Không tìm thấy gợi ý cho mã khách hàng {ma_khach_hang}.")
                continue

            # Lấy danh sách sản phẩm được gợi ý
            lst = []
            for row in user['recommendations']:
                row_f = df_san_pham_san_pham_idx.filter(df_san_pham_san_pham_idx.ma_san_pham_idx == row['ma_san_pham_idx'])
                row_f_first = row_f.first()
                lst.append((row['ma_san_pham_idx'], row_f_first['ma_san_pham'], row['rating']))

            dic_user_rec = {'ma_khach_hang': user.ma_khach_hang, 'recommendations': lst}

            # Chuyển danh sách gợi ý thành DataFrame
            recommended_products = spark.createDataFrame(
                dic_user_rec['recommendations'],
                schema=["ma_san_pham_idx", "ma_san_pham", "rating"]
            )

            # Kết hợp với products_sub để lấy tên và mô tả sản phẩm
            recommended_with_details = recommended_products.join(
                products_sub,
                on="ma_san_pham",
                how="left"
            )

            # Hiển thị kết quả
            print(f"Gợi ý sản phẩm cho khách hàng {ma_khach_hang}:")
            recommended_with_details.select("rating", "ma_san_pham", "ten_san_pham", "mo_ta").show(truncate=False)

        except ValueError:
            print("Vui lòng nhập một số nguyên hợp lệ.")


In [64]:
get_recommendations_with_details()

Nhập mã khách hàng (hoặc nhập 0 để thoát): 443
Gợi ý sản phẩm cho khách hàng 443:
+------------------+-----------+----------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Surprise

In [65]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357280 sha256=cfacdb4c697894f8b9318e282da1a7bc852b3a02045828fc89cef11370c1635e
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [66]:
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, CoClustering, BaselineOnly
from surprise.model_selection.validation import cross_validate

In [67]:
# Đổi PySpark DataFrame thành Pandas DataFrame
data_sub_pandas = data_sub.toPandas()

In [68]:
data_sub_pandas

,ma_san_pham,so_sao,ma_khach_hang
0,308500015,5.0,443
1,308500015,5.0,1030
2,422216594,5.0,689
3,204100075,5.0,2519
4,422208977,5.0,402
...,...,...,...
19624,422206043,5.0,1158
19625,100230057,5.0,760
19626,422201504,2.0,2248
19627,358300001,5.0,2886


In [69]:
# Đọc file San_pham.csv
products_pandas = pd.read_csv(path)

In [70]:
products_pandas

,ma_san_pham,ten_san_pham,gia_ban,gia_goc,phan_loai,mo_ta,diem_trung_binh
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,209000,435000.0,2x180ml\n180ml\nKhông Mùi\nCó Mùi Hương,Nước Hoa Hồng Klairs Supple Preparation là dòn...,4.8
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",147000,229000.0,2x400ml\n95ml\n400ml\nLàm Sạch Sâu\nTươi Mát D...,Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...,4.7
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,343000,455000.0,88ml\n236ml\n473ml\nDa khô/Hỗn hợp khô\nDa dầu...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...,4.9
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,377000,560000.0,2x50ml\n50ml,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...,4.6
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,210000,445000.0,20ml\n50ml,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...,4.6
...,...,...,...,...,...,...,...
1195,422219670,Serum Chống Nắng Dưỡng Thể Bioré Peach Pearly ...,185000,259000.0,Light To Medium\nMedium To Tan,Serum Chống Nắng Dưỡng Thể Bioré UV Aqua Rich ...,0.0
1196,226200001,Khăn Tẩy Trang Acnes 20 Miếng,53000,65000.0,NaN,Khăn Giấy Tẩy Trang Acnes Make Up Remover Shee...,3.5
1197,422218200,Mặt Nạ Colorkey Dưỡng Ẩm Sáng Da Với Niacinami...,11000,15000.0,1 miếng\n10 miếng\nDưỡng Ẩm Ngừa Nụn\nDưỡng Ẩm...,Mặt Nạ Colorkey Luminous B5 Facial Mask 25ml l...,4.0
1198,422206775,[Mua 1 Tặng 1] Freeplus Kem Dưỡng Ẩm Màng Nước...,899000,945000.0,NaN,Bộ sản phẩm bao gồm 2 món:\nKem Dưỡng Ẩm Màng ...,0.0


In [71]:
products_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ma_san_pham      1200 non-null   int64  
 1   ten_san_pham     1200 non-null   object 
 2   gia_ban          1200 non-null   int64  
 3   gia_goc          1186 non-null   float64
 4   phan_loai        678 non-null    object 
 5   mo_ta            1200 non-null   object 
 6   diem_trung_binh  1200 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 65.8+ KB


In [72]:
products_sub_pandas = products_pandas[['ma_san_pham', 'ten_san_pham', 'mo_ta']]

In [73]:
products_sub_pandas

,ma_san_pham,ten_san_pham,mo_ta
0,318900012,Nước Hoa Hồng Klairs Không Mùi Cho Da Nhạy Cảm...,Nước Hoa Hồng Klairs Supple Preparation là dòn...
1,205100137,"Nước Tẩy Trang L'Oreal Tươi Mát Cho Da Dầu, Hỗ...",Nước Tẩy Trang L'Oréal là dòng sản phẩm tẩy tr...
2,422208973,Sữa Rửa Mặt CeraVe Sạch Sâu Cho Da Thường Đến ...,Sữa Rửa Mặt Cerave Sạch Sâu là sản phẩm sữa rử...
3,204900013,Kem Chống Nắng La Roche-Posay Kiểm Soát Dầu SP...,Kem chống nắng giúp bảo vệ da khỏi tia UVB & U...
4,253900006,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm SPF 50...,Kem Chống Nắng Skin1004 Cho Da Nhạy Cảm là sản...
...,...,...,...
1195,422219670,Serum Chống Nắng Dưỡng Thể Bioré Peach Pearly ...,Serum Chống Nắng Dưỡng Thể Bioré UV Aqua Rich ...
1196,226200001,Khăn Tẩy Trang Acnes 20 Miếng,Khăn Giấy Tẩy Trang Acnes Make Up Remover Shee...
1197,422218200,Mặt Nạ Colorkey Dưỡng Ẩm Sáng Da Với Niacinami...,Mặt Nạ Colorkey Luminous B5 Facial Mask 25ml l...
1198,422206775,[Mua 1 Tặng 1] Freeplus Kem Dưỡng Ẩm Màng Nước...,Bộ sản phẩm bao gồm 2 món:\nKem Dưỡng Ẩm Màng ...


In [74]:
products_sub_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ma_san_pham   1200 non-null   int64 
 1   ten_san_pham  1200 non-null   object
 2   mo_ta         1200 non-null   object
dtypes: int64(1), object(2)
memory usage: 28.2+ KB


In [75]:
# Kiểm tra cột "ma_san_pham" trong products_sub_pandas
print("Giá trị không hợp lệ trong products_sub_pandas:")
print(products_sub_pandas[~products_sub_pandas['ma_san_pham'].astype(str).str.isdigit()])

Giá trị không hợp lệ trong products_sub_pandas:
Empty DataFrame
Columns: [ma_san_pham, ten_san_pham, mo_ta]
Index: []


In [76]:
reader = Reader(rating_scale=(1, 5))
data_suprise = Dataset.load_from_df(data_sub_pandas[['ma_khach_hang', 'ma_san_pham', 'so_sao']], reader)

In [77]:
data_suprise

## Viết hàm áp dụng thuật toán SVD, SVDpp, NMF, SlopeOne, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, CoClustering, BaselineOnly và gợi ý sản phẩm cho người dùng cụ thể khi nhập "ma_khach_hang".

In [78]:
def evaluate_model():
    # Danh sách các thuật toán
    algorithms = {
        "SVD": SVD(),
        "SVDpp": SVDpp(),
        "NMF": NMF(),
        "SlopeOne": SlopeOne(),
        "KNNBasic": KNNBasic(),
        "KNNBaseline": KNNBaseline(),
        "KNNWithMeans": KNNWithMeans(),
        "KNNWithZScore": KNNWithZScore(),
        "CoClustering": CoClustering(),
        "BaselineOnly": BaselineOnly()
    }

    # Lưu kết quả MAE của từng thuật toán
    mae_results = {}

    print("Đang chạy 5-fold cross-validation cho tất cả các thuật toán...\n")
    for name, algorithm in algorithms.items():
        print(f"Đang huấn luyện thuật toán {name}...\n")
        results = cross_validate(algorithm, data_suprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)
        print("\nKết quả cross-validation:")
        print(f"RMSE trung bình: {results['test_rmse'].mean():.4f}")
        mae_results[name] = results['test_mae'].mean()
        print(f"MAE trung bình: {mae_results[name]:.4f}\n")

    # Tìm thuật toán có MAE trung bình nhỏ nhất
    best_algorithm_name = min(mae_results, key=mae_results.get)
    best_algorithm = algorithms[best_algorithm_name]
    print(f"\nThuật toán có MAE trung bình nhỏ nhất: {best_algorithm_name}")
    print(f"MAE trung bình: {mae_results[best_algorithm_name]:.4f}\n")

    # Huấn luyện thuật toán tốt nhất với toàn bộ dữ liệu
    print(f"Đang huấn luyện thuật toán {best_algorithm_name} với toàn bộ dữ liệu...\n")
    trainset = data_suprise.build_full_trainset()
    best_algorithm = best_algorithm.fit(trainset)

    return best_algorithm

In [79]:
# Gọi hàm evaluate_and_recommend
best_algorithm = evaluate_model()

Đang chạy 5-fold cross-validation cho tất cả các thuật toán...

Đang huấn luyện thuật toán SVD...

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7920  0.7786  0.8344  0.8140  0.7763  0.7991  0.0222  
MAE (testset)     0.4817  0.4805  0.5017  0.4928  0.4749  0.4863  0.0096  
Fit time          0.65    0.59    0.82    1.10    0.73    0.78    0.18    
Test time         0.05    0.05    0.07    0.09    0.04    0.06    0.02    

Kết quả cross-validation:
RMSE trung bình: 0.7991
MAE trung bình: 0.4863

Đang huấn luyện thuật toán SVDpp...

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7867  0.7537  0.7123  0.7331  0.7406  0.7453  0.0247  
MAE (testset)     0.4426  0.4290  0.4121  0.4195  0.4292  0.4265  0.0103  
Fit time          0.93    0.92    0.91    1.15    1.02    0.99    0.09    
Test

In [80]:
def recommend_and_enrich_products(data_sub_pandas, products_sub_pandas, best_algorithm):
    # Tạo DataFrame rỗng để lưu top 5 kết quả gợi ý cho từng khách hàng
    enriched_recommendations_df = pd.DataFrame(columns=['ma_khach_hang', 'ma_san_pham', 'EstimateScore', 'ten_san_pham', 'mo_ta'])

    while True:
        try:
            # Nhập mã khách hàng
            ma_khach_hang = input("Nhập mã khách hàng (hoặc nhập 0 để thoát): ")
            ma_khach_hang = int(ma_khach_hang)  # Kiểm tra số nguyên

            if ma_khach_hang == 0:
                print("Thoát chương trình.")
                break  # Thoát khỏi vòng lặp

            # Tìm các sản phẩm mà người dùng đã đánh giá >= 3
            df_select = data_sub_pandas[(data_sub_pandas['ma_khach_hang'] == ma_khach_hang) & (data_sub_pandas['so_sao'] >= 3)]

            if df_select.empty:
                print(f"Khách hàng {ma_khach_hang} không có sản phẩm đã đánh giá >= 3.")
            else:
                print(f"Khách hàng {ma_khach_hang} đã đánh giá cao các sản phẩm sau:")
                print(df_select[['ma_san_pham', 'so_sao']].to_string(index=False))

            # Dự đoán điểm cho tất cả sản phẩm mà người dùng chưa đánh giá
            df_score = pd.DataFrame(data_sub_pandas['ma_san_pham'].unique(), columns=['ma_san_pham'])
            df_score['EstimateScore'] = df_score['ma_san_pham'].apply(
                lambda x: best_algorithm.predict(ma_khach_hang, x).est
            )

            # Lấy top 5 sản phẩm dựa trên EstimateScore
            top_5_df = df_score.sort_values(by=['EstimateScore'], ascending=False).head(5)
            top_5_df['ma_khach_hang'] = ma_khach_hang  # Thêm mã khách hàng

            # Kết hợp với thông tin sản phẩm từ products_sub_pandas
            enriched_top_5_df = pd.merge(
                top_5_df,
                products_sub_pandas,
                on='ma_san_pham',
                how='left'
            )

            # Hiển thị top 5 sản phẩm được gợi ý
            print(f"\nTop 5 sản phẩm được gợi ý cho khách hàng {ma_khach_hang}:")
            print(enriched_top_5_df[['ma_san_pham', 'EstimateScore', 'ten_san_pham', 'mo_ta']].to_string(index=False))

            # Lưu top 5 vào DataFrame tổng
            enriched_recommendations_df = pd.concat([enriched_recommendations_df, enriched_top_5_df], ignore_index=True)

        except ValueError:
            print("Vui lòng nhập một số nguyên hợp lệ.")

    # Lưu DataFrame tổng sau khi thoát vòng lặp
    print("\nKết quả gợi ý được lưu vào DataFrame enriched_recommendations_df:")
    print(enriched_recommendations_df.head())
    return enriched_recommendations_df

In [81]:
# Gọi hàm với dữ liệu và thuật toán tốt nhất đã được chọn
final_recommendations_df = recommend_and_enrich_products(data_sub_pandas, products_sub_pandas, best_algorithm)

Nhập mã khách hàng (hoặc nhập 0 để thoát): 443


<ipython-input-80-8d044974c690>:47: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  enriched_recommendations_df = pd.concat([enriched_recommendations_df, enriched_top_5_df], ignore_index=True)


Khách hàng 443 đã đánh giá cao các sản phẩm sau:
 ma_san_pham  so_sao
   308500015     5.0
   204100054     5.0
   422206537     5.0
   422202253     5.0
   422205162     5.0
   308500004     5.0
   308500018     5.0
   308500009     5.0
   422205164     5.0
   422205160     5.0
   204100019     5.0
   422205163     5.0
   308500037     5.0
   308500013     5.0

Top 5 sản phẩm được gợi ý cho khách hàng 443:
 ma_san_pham  EstimateScore                                                             ten_san_pham                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [82]:
# Hiển thị DataFrame kết quả cuối cùng
print(final_recommendations_df)

   ma_khach_hang ma_san_pham  EstimateScore  \
0            443   422205888       5.000000   
1            443   422211276       5.000000   
2            443   422216603       5.000000   
3            443   206400010       5.000000   
4            443   422221873       5.000000   
5           2027   422216602       5.000000   
6           2027   237800005       5.000000   
7           2027   224100029       5.000000   
8           2027   204000059       5.000000   
9           2027   422218099       5.000000   
10           736   208000001       5.000000   
11           736   204900009       5.000000   
12           736   248100018       5.000000   
13           736   213300001       5.000000   
14           736   335100004       5.000000   
15          4234   201600166       5.000000   
16          4234   422203173       5.000000   
17          4234   345800007       4.962050   
18          4234   345800008       4.962050   
19          4234   345800009       4.937112   

            

In [93]:
final_recommendations_df

,ma_khach_hang,ma_san_pham,EstimateScore,ten_san_pham,mo_ta
0,443,422205888,5.000000,Xịt Khoáng Cocoon Nghệ Hưng Yên Giúp Sáng Da 1...,Xịt Khoáng Cocoon Nghệ Hưng Yên Giúp Sáng Da 1...
1,443,422211276,5.000000,Kem Chống Nắng Naris Cosmetics Dạng Gel Dưỡng ...,Tham gia ngay chương trình Triệu phú Parasola:...
2,443,422216603,5.000000,Combo 2 Nước Tẩy Trang La Roche-Posay Dành Cho...,Nước Tẩy Trang La Roche-Posay Dành Cho Da Nhạy...
3,443,206400010,5.000000,Kem Dưỡng Olay Total Effects Dịu Nhẹ Ngừa Lão ...,Kem Dưỡng Olay Total Effects thuộc dòng sản ph...
4,443,422221873,5.000000,Combo 2 Sữa Chống Nắng Anessa Dưỡng Da Kiềm Dầ...,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...
5,2027,422216602,5.000000,Combo 2 Nước Tẩy Trang La Roche-Posay Cho Da D...,Nước Tẩy Trang Sạch Sâu & Kiểm Soát Nhờn Cho D...
6,2027,237800005,5.000000,Bông Tẩy Trang Mihoo Bông Tròn Túi 200 Miếng,Bông Tẩy Trang Mihoo Mermaid Cotton Pads Túi 1...
7,2027,224100029,5.000000,Dung Dịch Hada Labo Gokujyun Dưỡng Ẩm Da Tối Ư...,Dung Dịch Hada Labo Gokujyun Dưỡng Ẩm Tối Ưu 1...
8,2027,204000059,5.000000,Combo Xịt Khoáng Evoluderm Cấp Nước Cho Da 150...,Xịt Khoáng Cung Cấp Nước Evoluderm Atomiseur E...
9,2027,422218099,5.000000,Hộp 10 Miếng Mặt Nạ Wonjin Phục Hồi Chuyên Sâu...,Mặt Nạ Wonjin Effect Essence Mask 30g là sản p...


### Lưu mô hình best_algorithm

In [95]:
import pickle

# Lưu mô hình best_algorithm thành file pickle
with open('/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/best_algorithm.pkl', 'wb') as file:
    pickle.dump(best_algorithm, file)

print("Mô hình đã được lưu thành công vào file 'best_algorithm.pkl'")

Mô hình đã được lưu thành công vào file 'best_algorithm.pkl'


In [96]:
# Tải lại mô hình từ file pickle
with open('/content/gdrive/MyDrive/Colab Notebooks/DL07_K299_LyQuocHongPhuc/Project2/best_algorithm.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

print("Mô hình đã được tải lại thành công.")

Mô hình đã được tải lại thành công.


### Dự đoán cho nhiều sản phẩm chưa được đánh giá

In [97]:
def predict_products(data_sub_pandas, products_sub_pandas, best_algorithm):
    # Tạo DataFrame rỗng để lưu top 5 kết quả gợi ý cho từng khách hàng
    #enriched_recommendations_df = pd.DataFrame(columns=['ma_khach_hang', 'ma_san_pham', 'EstimateScore', 'ten_san_pham', 'mo_ta'])
    predict_products_df = pd.DataFrame(columns=['ma_khach_hang', 'ma_san_pham', 'EstimateScore', 'ten_san_pham', 'mo_ta'])

    while True:
        try:
            # Nhập mã khách hàng
            ma_khach_hang = input("Nhập mã khách hàng (hoặc nhập 0 để thoát): ")
            ma_khach_hang = int(ma_khach_hang)  # Kiểm tra số nguyên

            if ma_khach_hang == 0:
                print("Thoát chương trình.")
                break  # Thoát khỏi vòng lặp

            # Tìm các sản phẩm mà người dùng đã đánh giá >= 3
            df_select = data_sub_pandas[(data_sub_pandas['ma_khach_hang'] == ma_khach_hang) & (data_sub_pandas['so_sao'] >= 3)]

            if df_select.empty:
                print(f"Khách hàng {ma_khach_hang} không có sản phẩm đã đánh giá >= 3.")
            else:
                print(f"Khách hàng {ma_khach_hang} đã đánh giá cao các sản phẩm sau:")
                print(df_select[['ma_san_pham', 'so_sao']].to_string(index=False))

            # Dự đoán điểm cho tất cả sản phẩm mà người dùng chưa đánh giá
            df_score = pd.DataFrame(data_sub_pandas['ma_san_pham'].unique(), columns=['ma_san_pham'])
            df_score['EstimateScore'] = df_score['ma_san_pham'].apply(
                lambda x: loaded_model.predict(ma_khach_hang, x).est
            )

            # Lấy top 5 sản phẩm dựa trên EstimateScore
            top_5_df = df_score.sort_values(by=['EstimateScore'], ascending=False).head(5)
            top_5_df['ma_khach_hang'] = ma_khach_hang  # Thêm mã khách hàng

            # Kết hợp với thông tin sản phẩm từ products_sub_pandas
            enriched_top_5_df = pd.merge(
                top_5_df,
                products_sub_pandas,
                on='ma_san_pham',
                how='left'
            )

            # Hiển thị top 5 sản phẩm được gợi ý
            print(f"\nTop 5 sản phẩm được gợi ý cho khách hàng {ma_khach_hang}:")
            print(enriched_top_5_df[['ma_san_pham', 'EstimateScore', 'ten_san_pham', 'mo_ta']].to_string(index=False))

            # Lưu top 5 vào DataFrame tổng
            predict_products_df = pd.concat([predict_products_df, enriched_top_5_df], ignore_index=True)

        except ValueError:
            print("Vui lòng nhập một số nguyên hợp lệ.")

    # Lưu DataFrame tổng sau khi thoát vòng lặp
    print("\nKết quả gợi ý được lưu vào DataFrame enriched_recommendations_df:")
    print(predict_products_df.head())
    return predict_products_df

In [98]:
# Gọi hàm với dữ liệu và thuật toán tốt nhất đã được chọn
final_predict_df = predict_products(data_sub_pandas, products_sub_pandas, loaded_model)

Nhập mã khách hàng (hoặc nhập 0 để thoát): 443


<ipython-input-97-339f5ce63437>:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_products_df = pd.concat([predict_products_df, enriched_top_5_df], ignore_index=True)


Khách hàng 443 đã đánh giá cao các sản phẩm sau:
 ma_san_pham  so_sao
   308500015     5.0
   204100054     5.0
   422206537     5.0
   422202253     5.0
   422205162     5.0
   308500004     5.0
   308500018     5.0
   308500009     5.0
   422205164     5.0
   422205160     5.0
   204100019     5.0
   422205163     5.0
   308500037     5.0
   308500013     5.0

Top 5 sản phẩm được gợi ý cho khách hàng 443:
 ma_san_pham  EstimateScore                                                             ten_san_pham                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [99]:
final_predict_df

,ma_khach_hang,ma_san_pham,EstimateScore,ten_san_pham,mo_ta
0,443,422205888,5.000000,Xịt Khoáng Cocoon Nghệ Hưng Yên Giúp Sáng Da 1...,Xịt Khoáng Cocoon Nghệ Hưng Yên Giúp Sáng Da 1...
1,443,422211276,5.000000,Kem Chống Nắng Naris Cosmetics Dạng Gel Dưỡng ...,Tham gia ngay chương trình Triệu phú Parasola:...
2,443,422216603,5.000000,Combo 2 Nước Tẩy Trang La Roche-Posay Dành Cho...,Nước Tẩy Trang La Roche-Posay Dành Cho Da Nhạy...
3,443,206400010,5.000000,Kem Dưỡng Olay Total Effects Dịu Nhẹ Ngừa Lão ...,Kem Dưỡng Olay Total Effects thuộc dòng sản ph...
4,443,422221873,5.000000,Combo 2 Sữa Chống Nắng Anessa Dưỡng Da Kiềm Dầ...,Sữa Chống Nắng Anessa Perfect UV Sunscreen Ski...
5,2027,422216602,5.000000,Combo 2 Nước Tẩy Trang La Roche-Posay Cho Da D...,Nước Tẩy Trang Sạch Sâu & Kiểm Soát Nhờn Cho D...
6,2027,237800005,5.000000,Bông Tẩy Trang Mihoo Bông Tròn Túi 200 Miếng,Bông Tẩy Trang Mihoo Mermaid Cotton Pads Túi 1...
7,2027,224100029,5.000000,Dung Dịch Hada Labo Gokujyun Dưỡng Ẩm Da Tối Ư...,Dung Dịch Hada Labo Gokujyun Dưỡng Ẩm Tối Ưu 1...
8,2027,204000059,5.000000,Combo Xịt Khoáng Evoluderm Cấp Nước Cho Da 150...,Xịt Khoáng Cung Cấp Nước Evoluderm Atomiseur E...
9,2027,422218099,5.000000,Hộp 10 Miếng Mặt Nạ Wonjin Phục Hồi Chuyên Sâu...,Mặt Nạ Wonjin Effect Essence Mask 30g là sản p...


In [100]:
# Gọi hàm với dữ liệu và thuật toán tốt nhất đã được chọn
final_predict_df_1 = predict_products(data_sub_pandas, products_sub_pandas, loaded_model)

Nhập mã khách hàng (hoặc nhập 0 để thoát): 34


<ipython-input-97-339f5ce63437>:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predict_products_df = pd.concat([predict_products_df, enriched_top_5_df], ignore_index=True)


Khách hàng 34 đã đánh giá cao các sản phẩm sau:
 ma_san_pham  so_sao
   100230059     5.0
   422211446     5.0
   422211447     5.0
   318900012     5.0
   205100137     5.0
   422206648     5.0
   422216594     5.0
   205100146     5.0
   200400004     5.0
   422204259     5.0
   200400003     5.0
   100230057     5.0
   422208807     5.0
   318900011     5.0
   100230056     5.0
   205100136     5.0
   100230064     5.0

Top 5 sản phẩm được gợi ý cho khách hàng 34:
 ma_san_pham  EstimateScore                                                      ten_san_pham                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [101]:
final_predict_df_1

,ma_khach_hang,ma_san_pham,EstimateScore,ten_san_pham,mo_ta
0,34,335100004,5.000000,Kem Rửa Mặt Oxy Có Hạt Làm Sạch Sâu Giảm Nhờn ...,Kem Rửa Mặt OXY Deep Wash Scrub Formula là sản...
1,34,422201092,5.000000,Mặt Nạ Sur.Medic+ Cấp Nước & Cấp Ẩm Chuyên Sâu...,Mặt Nạ Sur.Medic+ Cấp Nước & Cấp Ẩm Chuyên Sâu...
2,34,422220466,5.000000,Combo 2 Nước Tẩy Trang Neutrogena Thanh Lọc Và...,Nước Tẩy Trang Neutrogena Thanh Lọc Và Làm Sạc...
3,34,422207562,5.000000,Combo 2 Nước Tẩy Trang Cocoon Bí Đao Làm Sạch ...,Nước Tẩy Trang Bí Đao Cocoon Winter Melon Mice...
4,34,422209722,5.000000,"Combo Mặt Nạ Banobagi Kháng Viêm, Giảm Mụn 30gx10",Banobagi Super Collagen Mask là dòng mặt nạ gi...
5,46,100210015,5.000000,"Serum Timeless Hyaluronic Acid Cấp Nước, Dưỡng...",Timeless Hyaluronic Acid Serum 100% Pure là dò...
6,46,208000001,5.000000,"Serum Timeless Hyaluronic Acid Cấp Nước, Dưỡng...",Timeless Hyaluronic Acid Serum 100% Pure là dò...
7,46,213300016,5.000000,Sữa Chống Nắng Bioré Bảo Vệ Hoàn Hảo - Sáng Mị...,Sữa Chống Nắng Bảo Vệ Hoàn Hảo Bioré UV Perfec...
8,46,422206798,5.000000,"Combo 3 Mặt Nạ Caryophy Giảm Mụn, Thâm & Dưỡng...","Mặt Nạ Caryophy Làm Giảm Mụn, Thâm & Dưỡng Ẩm ..."
9,46,422212871,5.000000,"Combo 10 Mặt Nạ Caryophy Giảm Mụn, Thâm & Dưỡn...","Mặt Nạ Caryophy Làm Giảm Mụn, Thâm & Dưỡng Ẩm ..."
